# Text Mining Project
### Retrieval
Creo degli embedding per il testo delle risposte e delle domande, poi addestro una rete che vada ad allineare lo spazio degli embedding delle domande con quello delle risposte. In questo modo, dato un testo di input, la rete mi restituisce l'embedding più simile tra quelli delle risposte (più similmente, le k risposte più simili). Questo embedding è poi utilizzato per trovare la risposta più simile tra quelle disponibili.
"Pre-computed answer embeddings served as ground true labels. In this way, quesion encoder was forced to learn projecting a question into an embedding close to answer embedding".

### Classification

#### Sviluppi futuri
Inserire la data di pubblicazione della domanda e della risposta, per poter compredere a valutare l'attualità della soluzione proposta.

In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
stack_data_raw = r"E:\J.A.R.V.I.S\stack_data\Codici Sorgente\codici_8(189602-200000).csv"
stack_data_df = pd.read_csv(stack_data_raw)
stack_data_df.head()

,id,codice sorgente
0,189602,"[<!DOCTYPE html>\n\n<html class=""html__respons..."
1,189603,"[<!DOCTYPE html>\n\n<html class=""html__respons..."
2,189604,"[<!DOCTYPE html>\n\n<html class=""html__respons..."
3,189605,"[<!DOCTYPE html>\n\n<html class=""html__respons..."
4,189606,"[<!DOCTYPE html>\n\n<html class=""html__respons..."


In [14]:
stack_data_df['answer'] = None
stack_data_df['tags'] = None

stack_data_df = stack_data_df.rename(columns={'id': 'header', 'codice sorgente': 'question'})
stack_data_df.head()

,header,question,answer,tags
0,189602,"[<!DOCTYPE html>\n\n<html class=""html__respons...",None,None
1,189603,"[<!DOCTYPE html>\n\n<html class=""html__respons...",None,None
2,189604,"[<!DOCTYPE html>\n\n<html class=""html__respons...",None,None
3,189605,"[<!DOCTYPE html>\n\n<html class=""html__respons...",None,None
4,189606,"[<!DOCTYPE html>\n\n<html class=""html__respons...",None,None


In [8]:
print(stack_data_df.shape)

(8935, 4)


### Getting relevant data from scraped web pages: 20k web pages --> 20m 47 sec

In [15]:
missing_info = 0

for idx,soup in enumerate(stack_data_df['question']):

    soup = BeautifulSoup(soup,"html.parser")

    try:
      # Getting the answer
      answer = soup.find('div', class_ = ['answer', 'js-answer'], itemprop='acceptedAnswer') 
      answer = answer.find('div', class_ = ['s-prose js-post-body'], itemprop='text')   
      stack_data_df['answer'][idx] = answer.text

    except:
      missing_info+=1
      continue
    
    # Getting the header
    header = soup.find('h1', class_ = ['fs-headline1 ow-break-word mb8 flex--item fl1'], itemprop ='name')
    stack_data_df['header'][idx] = header.text

    # Getting the question
    question = soup.find('div', class_ = ['question js-question'], id ='question') 
    question = question.find('div', class_ = ['s-prose js-post-body'], itemprop='text') 
    stack_data_df['question'][idx] = question.text

    # Getting the tags
    tags = soup.find('div', class_ = ['d-flex ps-relative fw-wrap'])
    tags = tags.text.split()[1:]
    # se è missing riempio con lista vuota
    if len(tags) == 0:
      tags = []
    stack_data_df['tags'][idx] =  tags

stack_data_df.to_csv(fr'C:\Users\kevin\Desktop\Text Mining data\df_21.csv', index = False)
print(f"In this scraping process we have {missing_info} of {idx} question/answer that were dropped because of missing information")

C:\Users\kevin\AppData\Local\Temp\ipykernel_50400\3869076605.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stack_data_df['answer'][idx] = answer.text
C:\Users\kevin\AppData\Local\Temp\ipykernel_50400\3869076605.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stack_data_df['header'][idx] = header.text


In this scraping process we have 19 of 10397 question/answer that were dropped because of missing information


## Contextual word embeddings

In [5]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import nltk

# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
texts = ["bank",
         "The river bank was flooded.",
         "The bank vault was robust.",
         "He had to bank on her for support.",
         "The bank was out of money.",
         "The bank teller was a man."]

In [47]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + str(text) + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [ ]:
# Getting embeddings for the target
# word in all given contexts
target_word_embeddings = []

# texts è una lista di stringhe
for text in texts:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    # Find the position 'bank' in list of tokens
    word_index = tokenized_text.index('bank')
    # Get the embedding for bank
    word_embedding = list_token_embeddings[word_index]

    target_word_embeddings.append(word_embedding)

In [60]:
print(len(tokenized_text), tokens_tensor.shape, segments_tensors.shape)

583 torch.Size([1, 583]) torch.Size([1, 583])


## Import data

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.head()

,header,question,answer,tags
0,How to find output instead of dimensions in ke...,"\nI tried to get the output of following code,...","\nFor that, use get_value:\nimport numpy as np...","['numpy', 'keras', 'deep-learning']"
1,"python, measure unique elements by row ignorin...",\nI have a data frame of 6 columns where each ...,\nYou can change myfunc to \ndef myfunc(row):\...,"['unique', 'counting']"
2,python division with large numbers automatical...,"\nPython 3.7.4 (tags/v3.7.4:e09359112e, Jul 8...",\nUse floor division:\n>>> 111111111111111111/...,['division']
3,speech recognition in PyCharm on Mac missing P...,"\n#raise AttributeError(""Could not find PyAudi...",\nYou may need to get some additional packages...,['pyaudio']
4,How to refer to a class list item?,\nI'm a beginner at Python and I don't know wh...,\ncommand should be set to a callback function...,"['tkinter', 'tk']"


In [51]:
df['answer'][0]

'\nFor that, use get_value:\nimport numpy as np\nimport keras\nfrom keras.layers import concatenate\nimport tensorflow as tf\n\na=np.array([1,2,3])\nb=np.array([4,5,6])\nc=np.array([7,8,9])\na = keras.backend.variable(a)\nb = keras.backend.variable(b)\nc = keras.backend.variable(c)\nmerged_vector = concatenate([a,b,c], axis=-1)\nprint(keras.backend.get_value(merged_vector))\n\n'

In [95]:
answer_embs = []
for idx,answer in enumerate(df['answer'][0:5000]):
    try:
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(str(answer), tokenizer)
        #print(len(tokenized_text), tokens_tensor.shape, segments_tensors.shape)
        list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
        centroid = np.mean(list_token_embeddings, axis=0)[None,]
        answer_embs.append(centroid.flatten())
    except Exception as e:
        pass     

In [96]:
from sklearn.neighbors import KDTree
import joblib
tree = KDTree(answer_embs) 

In [97]:
joblib.dump(tree, "kdtree_0_5000.joblib")

['kdtree_0_5000.joblib']

In [84]:
tree = joblib.load("cat_kdtree.joblib")
tree

24.369444444444447

In [87]:
dist, ind = tree.query(np.expand_dims(qfeatures, axis = 0), k= 1)

8480.565371024735

In [86]:
kimage.load_img(paths[ind[0][0]])

2400